In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

F:\public\anconda\envs\tf13\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\public\anconda\envs\tf13\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\public\anconda\envs\tf13\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\public\anconda\envs\tf13\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWa

1.13.1
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.18.1
pandas 0.24.2
sklearn 0.22.1
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
# print(fashion_mnist)
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]
# 打印格式
print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)


(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [3]:
print(np.max(x_train),np.min(x_train))

255 0


In [4]:
# 数据归一化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train:[None,28,28] -> [None,784]
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)

# 更改数据类型
y_train = np.asarray(y_train,dtype=np.int64)
y_valid = np.asarray(y_valid,dtype=np.int64)
y_test = np.asarray(y_test,dtype=np.int64)

In [5]:
print(np.max(x_train_scaled),np.min(x_train_scaled))

2.0231433 -0.8105136


In [6]:
# 创建dataset
def make_dataset(images,labels,epochs,batch_size,shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images,labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
        dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [9]:
batch_size = 20
epochs = 10

images_placeholder = tf.placeholder(tf.float32,[None,28*28])
labels_placeholder = tf.placeholder(tf.int64,[None,])

dataset = make_dataset(images_placeholder,labels_placeholder,epochs=epochs,batch_size=batch_size)

# 1. auto initialization
# 2. can't be re-initalized. make_initializable_iterator

dataset_iter = dataset.make_initializable_iterator()
x,y = dataset_iter.get_next()
with tf.Session() as sess:
    
    sess.run(dataset_iter.initializer,
            feed_dict = {
                images_placeholder:x_train_scaled,
                labels_placeholder:y_train
            }
            )
    
    x_val,y_val = sess.run([x,y])
    print(x_val.shape)
    print(y_val.shape)
    sess.run(dataset_iter.initializer,
            feed_dict = {
                images_placeholder:x_valid_scaled,
                labels_placeholder:y_valid
            })
    x_val,y_val = sess.run([x,y])
    print(x_val.shape)
    print(y_val.shape)

(20, 784)
(20,)
(20, 784)
(20,)


In [ ]:
hidden_units = [100,100]
class_num = 10

# x = tf.placeholder(tf.float32,[None,28*28])
# y = tf.placeholder(tf.int64,[None])
# 隐藏层
input_for_next_layer = x
for hidden_unit in hidden_units:
    input_for_next_layer = tf.layers.dense(input_for_next_layer,hidden_unit,activation=tf.nn.relu)

# 输出层
logits = tf.layers.dense(input_for_next_layer,class_num)
# last_hidden_output * w(logits) -> softmax -> pro
# 1. logit -> softmax -> prob
# 2. labels -> ont_hot
# 3. calculate cross entropy
loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=logits)

# get accuracy
prediction = tf.argmax(logits,1)
correct_prediction = tf.equal(prediction,y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

# 训练
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [ ]:
print(x)
print(logits)

In [ ]:
x_train.shape

In [ ]:
# session

init = tf.global_variables_initializer()
train_steps_per_epoch = x_train.shape[0] // batch_size

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):

            loss_val,accuracy_val,_ = sess.run(
                [loss,accuracy,train_op])
            print('\r[Train] epoch: %d,step: %d,loss:%3.5f,accuracy: %2.2f'%(epoch,step,loss_val,accuracy_val),end='')
